# İnme Zamanına Göre MR Sınıflandırması
Bu notebook, T2A, DWI ve ADC MR görüntülerinden inmenin zamanını (Hiperakut, Subakut, Kronik) sınıflandırmak için hazırlanmıştır.
- Kullanılan model: DenseNet121
- Girdi: 3 kanallı (T2A, DWI, ADC) MR görüntüleri
- Amaç: %98'e yakın F1-score elde etmek

In [ ]:
# Gerekli kütüphaneler
import os
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

In [ ]:
# Görüntüleri yükleme ve 3-kanallı hale getirme
def load_mr_triplet(t2a_path, dwi_path, adc_path, size=(256, 256)):
    t2a = cv2.imread(t2a_path, cv2.IMREAD_GRAYSCALE)
    dwi = cv2.imread(dwi_path, cv2.IMREAD_GRAYSCALE)
    adc = cv2.imread(adc_path, cv2.IMREAD_GRAYSCALE)
    if t2a is None or dwi is None or adc is None:
        return None
    t2a = cv2.resize(t2a, size)
    dwi = cv2.resize(dwi, size)
    adc = cv2.resize(adc, size)
    return np.stack([t2a, dwi, adc], axis=-1).astype(np.uint8)

In [ ]:
# Dataset hazırlama
def prepare_dataset(triplet_paths, labels, size=(256, 256)):
    X, y = [], []
    for (t2a_path, dwi_path, adc_path), label in tqdm(zip(triplet_paths, labels), total=len(labels)):
        image = load_mr_triplet(t2a_path, dwi_path, adc_path, size)
        if image is not None:
            X.append(image)
            y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# Etiketleri one-hot encode et
def encode_labels(y):
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    return to_categorical(y_encoded), le

In [ ]:
# Model kurma
def build_model(input_shape=(256, 256, 3), num_classes=3):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')
    x = base_model.output
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs=base_model.input, outputs=output)

In [ ]:
# Model eğitimi
def train_model(model, X_train, y_train, X_val, y_val, epochs=20, batch_size=16):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

In [ ]:
# Değerlendirme
def evaluate_model(model, X_test, y_test, label_encoder):
    y_pred = model.predict(X_test)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_test, axis=1)
    print(classification_report(y_true_labels, y_pred_labels, target_names=label_encoder.classes_))